In [1]:
import glob, json

In [2]:
json_file = glob.glob(pathname='*/*.json')
for j_file in json_file:
    with open(j_file, 'r', encoding='utf-8') as f:
        content = json.load(f)
        # content = json.dumps(content).replace(u'u2019', "'")
    # content_2 = json.dumps(content).replace("’", "'")
    if 'answers' in content:
        content['answers'] = [x for x in content['answers'] if x is not None]
    with open(j_file, 'w', encoding='utf-8') as f:
        json.dump(content, fp=f, indent=4, ensure_ascii=False)
    # with open(j_file, 'r', encoding='utf-8') as f:
    #     data = f.read()
    #     data = data.replace(u'\u2019', "'")
    # with open(j_file, 'w') as f:
    #     f.write(data)
    print(j_file)
    # break


Json file for web setup/Would-you-get-into-a-self-driving-car_0820_1.json
Json file for web setup/Should-I-invest-in-Bitcoin_0815_1.json


In [ ]:
with open('Json file for web setup/Should-I-invest-in-Bitcoin_0811_1.json', 'r', encoding='utf-8') as f:
    txt = f.read()
txt2 = txt.replace("’", "'").replace("“", "'").replace("”", "'")



In [ ]:
print(txt.replace("’", "'").replace("“", '"').replace("”", '"'))


In [37]:
txt2 = txt.replace("’", "'").replace("“", "'").replace("”", "'")


In [39]:
with open('x.json', 'w') as f:
    f.write(txt2)